In [121]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f

In [122]:
f = reload(f)
reload(dataset)
reload(eval)


<module 'eval' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\eval.py'>

In [63]:
# em numero de frases
#BATCH=30
BATCH=100 
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
#print(dicSentences_new_test[0])
print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))


sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])



BATCH: 100
Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Nega', 0, 422], ['sincope', 1, 427], ['.', 2, 434]], [['sincope', [1], 'Problema']]]
numero de sentencas no total: 100


In [64]:
tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))


idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 100


In [65]:
dic_predictions[9]

[[['Abd', 0],
  ['globoso', 1],
  [',', 2],
  ['flacido', 3],
  [',', 4],
  ['indolor', 5],
  ['a', 6],
  ['palpacao', 7],
  [',', 8],
  ['sem', 9],
  ['VCM', 10],
  ['.', 11]],
 [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]

In [56]:
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#all_predictions = f.load_obj('all_predictions_results_binarios_batch_'+str(BATCH))

In [66]:
for key, value in dic_predictions.items():
    print('key:',key)
    print(dic_predictions[key])
    if key>3:
        break

key: 0
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
key: 1
[[['Em', 0], ['acompanhamento', 1], ['no', 2], ['ambualtorio', 3], ['há', 4], ['5', 5], ['anos', 6], ['por', 7], ['FA', 8], [',', 9], ['uso', 10], ['de', 11], ['marevan', 12], ['5mg', 13], ['1', 14], ['x', 15], ['ao', 16], ['dia', 17], ['.', 18]], [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]
key: 2
[[['Comorbidades', 0], [':', 1], ['DM', 2], ['há', 3], ['10', 4], ['anos', 5], ['em', 6], ['uso', 7], ['de', 8], ['metformina', 9], ['850mg', 10], ['3', 11], ['cp', 12], ['/', 13], ['dia', 14], [',', 15], ['acarbose', 16], ['1', 17], ['cp', 18], ['/', 19], ['dia', 20], ['e', 21], ['glicazida', 22], ['60mg', 23], ['2', 24], ['cp', 25], ['/', 26], ['dia', 27], ['e', 28], ['insulina', 29], ['(', 30], ['24', 31], ['-', 32], ['0', 33], ['-', 34], ['24', 35], [')', 36], ['.', 37]], [['Comorbidades', [0], 'Problema'], ['DM', [2], 'Problema'], ['metformina 850mg', [9, 10], 'Tratamento'], ['acarbose'

In [67]:
dic_predictions[1][1]

[['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]

In [68]:
print(len(dicSentences_new_test))
print(len(dic_predictions))

100
100


In [69]:
region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)


In [70]:
region_pred_list[:4]

['Problema', 'Tratamento', 'Problema', 'Problema']

In [71]:
region_true_list[:4]

['Problema', 'Tratamento', 'Problema', 'Problema']

In [72]:
lista_erros[:8]

[7, 8, 13, 13, 14, 15, 15, 15]

In [73]:
print(len(lista_erros))
print(len(set(lista_erros)))

114
45


In [74]:
print(dic_predictions[8])
print(dicSentences_new_test[8][1])

[[['BC', 0], ['arritmicas', 1], [',', 2], ['NF', 3], ['SS', 4], ['2T', 5], ['.', 6]], [['SS', [4], 'Problema']]]
[['BC arritmicas', [0, 1], 'Problema'], ['SS', [4], 'Problema']]


In [75]:
print(dic_predictions[11])
print(dicSentences_new_test[11][1])

[[['Exames', 0], ['-', 1], ['Holter', 2], [':', 3], ['FC', 4], ['controlada', 5], [',', 6], ['media', 7], ['92', 8], ['.', 9]], [['Exames', [0], 'Teste'], ['Holter', [2], 'Teste']]]
[['Exames', [0], 'Teste'], ['Holter', [2], 'Teste']]


In [76]:
print(dic_predictions[13])
print(dicSentences_new_test[13][1])

[[['Ecocardiograma', 0], ['-', 1], ['ventrículo', 2], ['esquerdo', 3], ['com', 4], ['hipertrofia', 5], ['concentrica', 6], ['de', 7], ['grau', 8], ['discreto', 9], ['e', 10], ['função', 11], ['sistólica', 12], ['preservada', 13], ['.', 14]], [['Ecocardiograma', [0], 'Teste'], ['ventrículo esquerdo', [2, 3], 'Anatomia'], ['hipertrofia concentrica de grau discreto', [5, 6, 7, 8, 9], 'Problema']]]
[['Ecocardiograma', [0], 'Teste'], ['ventrículo esquerdo com hipertrofia concentrica de grau discreto', [2, 3, 4, 5, 6, 7, 8, 9], 'Problema'], ['ventrículo esquerdo', [2, 3], 'Anatomia']]


In [77]:
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))




len(region_true_list): 352
len(region_pred_list): 352
-----Avaliando só modelo de NER:-----
              precision    recall  f1-score   support

    Anatomia   0.789474  0.277778  0.410959        54
           O   0.000000  0.000000  0.000000        38
    Problema   0.853211  0.823009  0.837838       113
       Teste   0.880952  0.891566  0.886228        83
  Tratamento   0.828125  0.828125  0.828125        64

    accuracy                       0.667614       352
   macro avg   0.670352  0.564096  0.592630       352
weighted avg   0.753305  0.667614  0.691546       352

[[15 36  1  1  1]
 [ 4  0 15  9 10]
 [ 0 20 93  0  0]
 [ 0  9  0 74  0]
 [ 0 11  0  0 53]]


### Agora, modelo de SpanClassification

In [78]:
print('-----Avaliando só com modelo de Spam:-----')
model = f.getModel()

-----Avaliando só com modelo de Spam:-----
loading model from C:\Users\lisat\OneDrive\jupyter notebook\span-model\model-exp3
num_labels: 5


Some weights of the model checkpoint at C:\Users\lisat\OneDrive\jupyter notebook\span-model\model-exp3 were not used when initializing BertForChunkClassification: ['classifier.0.weight', 'classifier.0.bias']
- This IS expected if you are initializing BertForChunkClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForChunkClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForChunkClassification were not initialized from the model checkpoint at C:\Users\lisat\OneDrive\jupyter notebook\span-model\model-exp3 and are newly initialized: ['classifier.1.weight', 'classifier.1.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [79]:
dic_sentencesTrainDev = f.load_obj('dic_sentencesTrainDev')
dicPosTagger, _ = f.getDicPosTagger(dic_sentencesTrainDev)


In [85]:
lista_postaggers_entidades = f.getListaPostaggerEntidades(dic_predictions, dicPosTagger)


In [86]:
dic_predictions[1][1][0]

['FA', [8], 'Problema']

In [133]:
# para gerar arquivo de predicoes (com as tags <e1>)
combinacaoEntidadesAll, pullando_termos_pos_tagger = f.getCombinacaoEntidades(dic_predictions, True, dicPosTagger, lista_postaggers_entidades)
f.gravaArquivoPredict(combinacaoEntidadesAll)

Com filtro-postagger
len(combinacaoEntidadesAll:) 100
len(pulando_termos_postagger): 549
numTotalEntidades: 455
numFrases: 86
gravando em: data/span.predict


In [128]:
pullando_termos_pos_tagger[:15]

[[['FA', ','], '-N-PU'],
 [['Comorbidades', ':'], '-N-PU'],
 [['insulina', '('], '-N-PU'],
 [['losartana', '50mg', '/'], '-N-N-PU'],
 [['50mg', '/'], '-N-PU'],
 [['carvedilol', '25', '12'], '-N-NUM-NUM'],
 [['25', '12'], '-NUM-NUM'],
 [['HCTZ', '.'], '-N-PU'],
 [['DSLP', 'em'], '-N-PREP'],
 [['sinvastatina', ','], '-N-PU'],
 [['dispnéia', ','], '-N-PU'],
 [['DPN', 'e'], '-N-KC'],
 [['ortopneia', ','], '-N-PU'],
 [['palpitações', 'e'], '-N-KC'],
 [['sincope', '.'], '-N-PU']]

In [129]:
print(combinacaoEntidadesAll[1])


['Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .', [['Em acompanhamento no ambualtorio há 5 anos por <e1> FA </e1> , uso de marevan 5mg 1 x ao dia .', 'Problema'], ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de <e1> marevan 5mg </e1> 1 x ao dia .', 'Tratamento'], ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de <e1> marevan </e1> 5mg 1 x ao dia .', 'O'], ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan <e1> 5mg </e1> 1 x ao dia .', 'O']]]


In [146]:
# chamar predict                
pred_region_labels = f.predictSpan(model)
print('len(pred_region_labels):', len(pred_region_labels))


os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
LOOKING AT ./data\span.predict
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
*** Example ***
tokens: ['[CLS]', 'Em', 'ac', '##om', '##pan', '##ham', '##ento', 'no', 'amb', '##ual', '##torio', 'há', '5', 'anos', 'por', 'FA', ',', 'uso', 'de', 'mare', '##van', '5', '##m', '##g', '1', 'x', 'ao', 'dia', '.', '[SEP]']
lista_label: []
indices_e1_sentenca: []
*** Example ***
tokens: ['[CLS]', 'Como', '##rb', '##idades', ':', 'DM', 'há', '10', 'anos', 'em', 'uso', 'de', 'met', '##form', '##ina', '850', '##m', '##g', '3', 'c', '##p', '/', 'dia', ',', 'ac', '##ar', '##bos', '##e', '1', 'c', '##p', '/', 'dia', 'e', 'gli', '##ca', '##zid', '##a', '60', '##m', '##g', '2', 'c', '##p', '/', 'dia', 'e', 'ins', '##ulin', '##a', '(', '24', '-', '0', '-', '24', ')', '.', '[SEP]']
lista_label: []
indices_e

In [ ]:
f.save_obj('combinacaoEntidadesAll_'+str(BATCH), combinacaoEntidadesAll)


In [153]:
f = reload(f)
reload(dataset)
reload(eval)


<module 'eval' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\eval.py'>

In [157]:
pred_region_labels[:50]

tensor([3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3,
        1, 3, 4, 3, 3, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3,
        3, 3])

In [155]:
#combinacaoEntidadesAll = load_obj(combinacaoEntidadesAll_'+str(BATCH))

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)

dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)



combinacao: []
combinacao: ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .', [['Em acompanhamento no ambualtorio há 5 anos por <e1> FA </e1> , uso de marevan 5mg 1 x ao dia .', 'Problema'], ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de <e1> marevan 5mg </e1> 1 x ao dia .', 'Tratamento'], ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de <e1> marevan </e1> 5mg 1 x ao dia .', 'O'], ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan <e1> 5mg </e1> 1 x ao dia .', 'O']]]
combinacao: ['Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .', [['<e1> Comorbidades </e1> : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .', 'Problema'], ['Comorbidades : <e1> DM </e1> há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicaz

IndexError: index 449 is out of bounds for dimension 0 with size 449

In [ ]:

# tbm sem o postagger pra ver

print('-----Avaliando os dois modelos juntos:-----')

# dic_predictions_all = mesclando o resultado dos dois modelos (dic_predictions + combinacaoEntidadesAll_pred)
dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)
f.save_obj('dic_predictions_all_io_bkp', dic_predictions_all)
#dic_predictions_all = load_obj('dic_predictions_all_io_bkp')

In [137]:
# juntar all_predictions
dic_predictions = {}
for num, prediction in enumerate(all_predictions):
    if num==0:
        dic_predictions = prediction.copy()
        continue
    for key, value in prediction.items():
        entidadesJaEstavam=dic_predictions[key][1]
        tokens=dic_predictions[key][0]
        lista_entidade = [e for e in entidadesJaEstavam]
        #print('lista_entidade:', lista_entidade)
        entidades = value[1].copy()
        if len(entidades)>0:
            #print(entidades)
            for entidade in entidades:
                lista_entidade.append(entidade)
            dic_predictions[key]=[tokens,lista_entidade]
